In [1]:
import nltk
import os
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.tag import StanfordPOSTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
java_path = "C:/Program Files/Java/jdk1.8.0_144/bin/java.exe"
os.environ['JAVAHOME'] = java_path
model='stanford-postagger-2018-02-27/models/english-left3words-distsim.tagger'
jar='stanford-postagger-2018-02-27/stanford-postagger-3.9.1.jar'
st=StanfordPOSTagger(model,jar,encoding='utf-8')
lemmatizer=WordNetLemmatizer()

c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\nltk\tag\stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


In [2]:
senti_words = []
f=open("D:/Lex/SentiWordNet_3.0.0_20130122.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[4]
    #print(w)
    i=w.index('#')
    #print(i)
    senti_words.append(w[:i])


In [3]:
senti_words = []
f=open("improvedsamplelexicon.txt","r",encoding="utf-8") 
all_lines=f.readlines()
for line in all_lines:
    
    w=line.split('\t')[1]
    senti_words.append(w)

In [4]:
senti_words.index('hoax')

3794

In [5]:
neg= {"aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
 "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
 "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
 "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
 "neednt", "needn't", "never","no" "none", "nope", "nor", "not", "nothing", "nowhere",
 "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
 "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
 "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite","unfulfilled","undue","anti"}

In [18]:
boost={"absolutely":0.5, "amazingly":0.125, "awfully":0.25, "completely":0.25, "considerably":0.125,
 "decidedly":0.25 , "deeply":0.25 , "enormously":0.25 ,
 "entirely":0.5 , "especially":0.25 , "exceptionally":0.25, "extremely":0.625 ,"fabulously":0.25   , "fully":0.375  ,
 "greatly":0.125 , "highly":0.5   ,"huge":0.25, "hugely":0.25   , "incredibly":0.25   ,
 "intensely":0.25   , "majorly":0.625  ,"particularly":0.125 ,"really":0.375 ,"substantially":0.125   ,
 "thoroughly":0.625   , "totally":0.5  ,"unbelievably":0.25   , "utterly":0.5   ,
 "very":0.25  ,"tremendously":-0.25 ,  "barely":-0.375   ,"badly":-0.125 ,"hardly":-0.25,"unusually":-0.5   , 
 "less":-0.5   , "little":-0.375   , "marginally":-0.125   , "partly":-0.125   ,
 "scarcely":-0.25   , "slightly":-0.25   , "somewhat":-0.125, "shoddy":-0.625, "poorly":-0.75}
pos_boost=["absolutely", "amazingly", "awfully", "completely", "considerably",
 "decidedly","deeply","enormously","entirely","especially","exceptionally","extremely","fabulously","fully",
 "greatly","highly","huge","hugely", "incredibly","intensely","majorly","particularly","really","substantially","thoroughly","totally","unbelievably", "utterly","very"]
neg_boost=["tremendously", "barely","badly","hardly","unusually", "less", "little", "marginally", "partly",
 "scarcely", "slightly","somewhat", "shoddy", "poorly"]

In [19]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [20]:
def find_sentiment(word,pos):
    found=0
    line_nos=[index for index, value in enumerate(senti_words) if value == word]
    lines=[all_lines[lno] for lno in line_nos]
    if len(lines)==0:
        return None
    for l in lines:
        if l[0]==pos:
            pos_score,neg_score=l.split('\t')[2],l.split('\t')[3]
            found=1
            break
        else:
            continue
    if found==1:
        return(float(pos_score)-float(neg_score))
    else:
        pos_score,neg_score=lines[0].split('\t')[2],lines[0].split('\t')[3]
        return(float(pos_score)-float(neg_score))

In [30]:
def get_sentiment(word,pos):
    second_sent=find_sentiment(word,pos)
            
    if second_sent is None:
        try:
            lemma=lemmatizer.lemmatize(word,pos)
            synsets = wn.synsets(lemma)
            synset=synsets[0]
            s=synset.name()
            i=s.index('.')
            second_sent=find_sentiment(s[:i],pos)
            #print(s[:i],second_sent)
            if second_sent is None:
                second_sent=0.0
        except:
            second_sent=0.0
    return second_sent

In [31]:
def normalize(score):
    return((score-(-1.875))/3.125)

In [32]:
nouns=open('sent_nouns.txt','r',encoding='utf-8')
nouns=nouns.read()
nouns=nouns.split()

In [33]:
punctuations = '''!@'#-+='''
def clean_tweet(t):
    for c in punctuations:
             t= t.replace(c,"")
    return t

In [34]:
def positive_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(boost[booster]+sec_score)
    if sec_score<0.0:
        return(sec_score-boost[booster])
    else:
        return(boost[booster])

In [35]:
def negative_booster(booster,word,pos2):
    sec_score=get_sentiment(word,pos2)
    if sec_score>0.0:
        return(sec_score+boost[booster])
    if sec_score<0.0:
        return(sec_score+boost[booster])
    else:
        return(boost[booster])

In [27]:
tweet1='''The marginalization of the Tax Cheats which was expected post #GST will now happen in right earnest as #Ewaybill rolls out. Real benefits to most organized sector companies will be seen in FY2019'''
tweet2='''From Failed #Demonetisation to Shoddy Implementation of #GST to poorly performing #Economy to unfulfilled Promise of reduction in fuel prices to not keeping the promises of providing 2 crores jobs, BJP has wrecked a nation & has broken people’s faith '''
tweet3='''There is nothing wrong with #GST implementation but the fact is we are all losers. Injected to us by @INCIndia  Even God cannot change'''
tweet4='''So, Indian businesses are mad against GST cause now they have to pay tax and earlier they weren't?'''
tweet5='''#GST in india is total failure,it like nightmare for tax payers Indians..in my view it could be rolled out after #2019LSPolls after corrections ~ Dr Subramanian @Swamy39 Speaking at 14th Annual 🇮🇳India Business Conference on ‘Indian Growth @Columbia_Biz '''
tweet6='''#GST complexities and high tax slabs is really killing small-businesses in India. It only aims at ultimately benefitting the big businessmen.'''

In [28]:
A=word_tokenize(clean_tweet(tweet6.lower()))
bigrams=nltk.ngrams(A,2)

In [29]:
i=0
not_found=set()
score=0.0
for b in bigrams:
    i+=1
    b_tag=st.tag(b)
    print(b_tag)
    second_gram=b_tag[1]
    first_gram=b_tag[0]
    if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
    #if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VB'):
        pos2=convert_tag(second_gram[1])
        pos1=convert_tag(first_gram[1])
        
        if first_gram[0] in boost:
            if first_gram[0] in pos_boost:
                score=positive_booster(first_gram[0],second_gram[0],pos2)
                print('Pos-Boost:',b_tag,score)
            if first_gram[0] in neg_boost:
                score=negative_booster(first_gram[0],second_gram[0],pos2)
                print('Neg-Boost:',b_tag,score)
            
   
        if first_gram[0] in neg:
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    score+=(get_sentiment(first_gram[0],pos1)+sec_score)
                else:
                    score+=-(sec_score)
                print('Neg:',b_tag,score)
            
        elif first_gram[0] not in neg and first_gram[0] not in boost:
            try:
                first_score=get_sentiment(first_gram[0],pos1)
                score+=first_score+get_sentiment(second_gram[0],pos2)
                print('Normal:',b_tag,score)
            except:
                print("Not Found:",second_gram[0])
                not_found.add(second_gram[0])
    
print(normalize(score))

           

[('gst', 'NN'), ('complexities', 'NNS')]
complexity None
Normal: [('gst', 'NN'), ('complexities', 'NNS')] 0.0
[('complexities', 'NNS'), ('and', 'CC')]
[('and', 'CC'), ('high', 'JJ')]
Normal: [('and', 'CC'), ('high', 'JJ')] 0.125
[('high', 'JJ'), ('tax', 'NN')]
[('tax', 'NN'), ('slabs', 'NNS')]
[('slabs', 'NNS'), ('is', 'VBZ')]
[('is', 'VBZ'), ('really', 'RB')]
[('really', 'RB'), ('killing', 'VBG')]
Pos-Boost: [('really', 'RB'), ('killing', 'VBG')] -0.625
[('killing', 'VBG'), ('smallbusinesses', 'NNS')]
[('smallbusinesses', 'NNS'), ('in', 'IN')]
[('in', 'IN'), ('india', 'NN')]
[('india', 'NN'), ('.', '.')]
[('.', '.'), ('it', 'PRP')]
[('it', 'PRP'), ('only', 'RB')]
[('only', 'RB'), ('aims', 'VBZ')]
[('aims', 'NNS'), ('at', 'IN')]
[('at', 'IN'), ('ultimately', 'RB')]
[('ultimately', 'RB'), ('benefitting', 'VBG')]
benefit 0.375
Normal: [('ultimately', 'RB'), ('benefitting', 'VBG')] -0.25
[('benefitting', 'VBG'), ('the', 'DT')]
[('the', 'DT'), ('big', 'JJ')]
Normal: [('the', 'DT'), ('big',

In [18]:
def check(bigrams):
    score=0.0
    for b in bigrams:
        #b_tag=st.tag(b)
        b_tag=pos_tag(b)
        #print(b_tag)
        second_gram=b_tag[1]
        first_gram=b_tag[0]
        if second_gram[0] in nouns or second_gram[1].startswith('J') or second_gram[1].startswith('RBR') or second_gram[1].startswith('RBS') or second_gram[1].startswith('VBG') or second_gram[1].startswith('VBN') or second_gram[1].startswith('VBD'):
            pos2=convert_tag(second_gram[1])
            pos1=convert_tag(first_gram[1])
        
            if first_gram[0] in boost:
                if first_gram[0] in pos_boost:
                    score=positive_booster(first_gram[0],second_gram[0],pos2)
                    print('Pos-Boost:',b_tag,score)
                if first_gram[0] in neg_boost:
                    score=negative_booster(first_gram[0],second_gram[0],pos2)
                    print('Neg-Boost:',b_tag,score)
            
   
            if first_gram[0] in neg:
                x=0.0
                sec_score=get_sentiment(second_gram[0],pos2)
                print(second_gram[0],sec_score)
                if sec_score ==0.0 or sec_score==None:
                    try:
                        x=get_sentiment(first_gram[0],pos1)
                        score+=(x+sec_score)
                    except:
                        print(x)
                else:
                    score+=-(sec_score)
                print('Neg:',b_tag,score)
            
            elif first_gram[0] not in neg and first_gram[0] not in boost:
                try:
                    first_score=get_sentiment(first_gram[0],pos1)
                    score+=first_score+get_sentiment(second_gram[0],pos2)
                    print('Normal:',b_tag,score)
                except:
                    print("Not Found:",second_gram[0])
                    #not_found.add(second_gram[0])
    return (score)
    
    

In [19]:
fw=open("tweets.txt","r",encoding="utf-8")
rw=open("nltk_tweet_result.txt","a",encoding='utf-8')
tw=fw.read().split('\n')
i=0
for t in tw:
    i+=1
    score=0.0
    tok_tweet=word_tokenize(clean_tweet(t.lower()))
    bigrams=nltk.ngrams(tok_tweet,2)
    score=check(bigrams)
    rw.write(str(t)+"  :"+str(score)+"\n")
    print(i)
fw.close()
rw.close()

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
1
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
2
3
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
4
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
5
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
6
<class 'float'>
<class 'float'>
7
<class 'float'>
<class 'float'>
8
<class 'float'>
<class 'float'>
<class 'float'>
9
<class 'float'>
<class 'float'>
<class 'float'>
10
<class 'float'>
11
<class 'float'>
Neg-Boost: [('badly', 'RB'), ('affected', 'VBD')] -0.625
<class 'float'>
12
<class 'float'>
<class 'float'>
13
<class 'float'>
14
<class 'float'>
<class 'float'>
<class 'float'>
15
<class 'float'>
<class 'float'>
16
<class 'float'>
<class 'float'>
17
<class 'float'>
<class 'float'>
<class 'float'>
18
<class 'float'>
<class 'float'>
<class 'float'>
19
<class 'float'>
20
<class 'float'>
21
<class 'float'>
<class 'f

168
<class 'float'>
<class 'float'>
169
170
<class 'float'>
171
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
172
173
<class 'float'>
<class 'float'>
174
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
175
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
176
177
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
178
<class 'float'>
<class 'float'>
<class 'float'>
179
<class 'float'>
<class 'float'>
180
181
<class 'float'>
<class 'float'>
182
<class 'float'>
183
<class 'float'>
Neg-Boost: [('little', 'JJ'), ('hard', 'JJ')] -1.125
<class 'float'>
<class 'float'>
<class 'float'>
184
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
185
<class 'float'>
<class 'float'>
<class 'float'>
186
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
187
<class 'float'>
<class 'float'>
<class 'float'>
188
<class 'float'>
<class 'float'>
189
<class 'flo

In [108]:
fo=open("stan_tweet_result.txt","r",encoding='utf-8')
lines=fo.readlines()
s_list=[]
for l in lines:
    s_list.append(float(l.split(':')[1]))

In [125]:
pos_list,neg_list=[],[]
for s in s_list:
    if s >=0.0:
        pos_list.append(s)
    if s<0.0:
        neg_list.append(-s)

In [126]:
max(neg_list

[0.06727784728718064,
 0.625,
 1.25,
 0.378,
 0.5,
 0.125,
 0.25,
 0.625,
 0.5,
 1.375,
 0.625,
 0.03582397526189089,
 0.25,
 0.125,
 0.125,
 0.75,
 0.5,
 1.375,
 0.03582397526189089,
 0.25,
 1.381,
 0.125,
 0.25,
 0.25,
 0.75,
 1.25,
 0.875,
 0.06727784728718064,
 0.625,
 1.125,
 0.625,
 1.375,
 0.375,
 1.375,
 0.75,
 1.5,
 0.5,
 0.75,
 0.375,
 0.375,
 0.375,
 0.375,
 0.5,
 0.875,
 0.378,
 1.125,
 1.125,
 1.875,
 0.5,
 0.03582397526189089,
 0.625,
 0.25,
 0.125,
 0.375,
 0.25,
 1.375,
 0.125,
 0.25,
 0.03582397526189089,
 1.0,
 0.0358239752618908,
 0.375,
 0.03582397526189089,
 0.125,
 0.625,
 0.125,
 0.125,
 0.25,
 0.625,
 0.5,
 0.25]

In [442]:
word='loot'
lemma = lemmatizer.lemmatize(word)            
synsets = wn.synsets(lemma)
synset = synsets[0]
swn_synset = swn.senti_synset(synset.name())
print(swn_synset.pos_score()) 
print(swn_synset.neg_score())

0.0
0.0


In [443]:
lemma

'loot'

In [444]:
s=synset.name()
i=s.index('.')
s[:i]

'loot'

In [445]:
if word in senti_words:
    line_nos=[index+1 for index, value in enumerate(senti_words) if value == word]
    line_no=all_lines[senti_words.index(word)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
            

elif lemma in senti_words: 
    line_no=lines[senti_words.index(lemma)]
    pos_score,neg_score=line_no.split('\t')[2],line_no.split('\t')[3]
    

In [250]:
[index+1 for index, value in enumerate(senti_words) if value == 'mad']

[14272]

In [252]:
line_no

'a\t02572038\t0.375\t0.25\tmad#4 insane#2 harebrained#1\tvery foolish; "harebrained ideas"; "took insane risks behind the wheel"; "a completely mad scheme to build a bridge between two mountains"\n'

In [523]:
a='''Demonetization + GST is the biggest reset Indian Economy has witnessed since 1991. Yes...disruptive, chaotic but was necessary '''
a=word_tokenize(a.lower())

pos_tag(a)

[('demonetization', 'NN'),
 ('+', 'NNP'),
 ('gst', 'NN'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('biggest', 'JJS'),
 ('reset', 'NN'),
 ('indian', 'JJ'),
 ('economy', 'NN'),
 ('has', 'VBZ'),
 ('witnessed', 'VBN'),
 ('since', 'IN'),
 ('1991.', 'CD'),
 ('yes', 'NNS'),
 ('...', ':'),
 ('disruptive', 'NN'),
 (',', ','),
 ('chaotic', 'JJ'),
 ('but', 'CC'),
 ('was', 'VBD'),
 ('necessary', 'JJ')]